In [169]:
import pandas as pd

In [170]:
result_file = 'saved_models/ranks_test_evaluation_ComplEx_0.2_0.0_literal_Synthetic_numerical_literals_train.tsv'
dataset_name = 'Synthetic'

# should not be modified
value_relation = '/m/is_a'
category_a = '/m/high'
category_b = '/m/low'

In [189]:
# read the results (only value_relation triples are relevant for the evaluation, therefore we filter the dataframe)
df_results = pd.read_csv(result_file, sep='\t', header=None,  names=["head", "relation", "tail", "rank head", "rank tail", "score"])
df_results = df_results[df_results['relation']==value_relation]

In [190]:
# read the true literal values that are used to classify into the two categories (the file should only contain one literal relation type)
df_literals = pd.read_csv(f'data/{dataset_name}/literals/numerical_literals.txt', sep='\t', header=None,  names=["head", "relation", "literal"])

In [219]:
# derive human entities from the class mapping file
class_mapping_df = pd.read_csv('FB15k-237_class_mapping.csv', sep=';', header=0)
human_uris = class_mapping_df[class_mapping_df["class_label"] == "human"]["dataset_entity"]

In [222]:
df_literals[df_literals['head'].isin(human_uris.to_list())]

,head,relation,literal
3,/m/01sl1q,/m/has_value,0.284997
6,/m/07nznf,/m/has_value,0.414713
8,/m/0q9kd,/m/has_value,0.379819
15,/m/04bdxl,/m/has_value,0.303381
17,/m/079vf,/m/has_value,0.983388
...,...,...,...
13503,/m/09f5pp,/m/has_value,0.790471
13647,/m/0glyyw,/m/has_value,0.686262
13704,/m/06rkl,/m/has_value,0.418615
13717,/m/02p59ry,/m/has_value,0.224405


In [223]:
def get_scores_for_certain_tail(tail_entity):
    related_scores = pd.merge(df_literals[df_literals['head'].isin(human_uris.to_list())], df_results[df_results["tail"]==tail_entity][['head', 'score']], left_on='head', right_on='head')
    related_scores = related_scores.rename(columns={ 'score': f'score {tail_entity}' })
    return related_scores

In [225]:
a = get_scores_for_certain_tail(category_a)
b = get_scores_for_certain_tail(category_b)

merged = pd.merge(a, b)

In [228]:
merged

,head,relation,literal,score /m/high,score /m/low
0,/m/079vf,/m/has_value,0.983388,0.581661,0.724532
1,/m/05d7rk,/m/has_value,0.052213,0.341893,0.357488
2,/m/03qcq,/m/has_value,0.221566,0.615127,0.912245
3,/m/01k7d9,/m/has_value,0.565050,0.659364,0.248403
4,/m/06dv3,/m/has_value,0.711868,0.147361,0.917865
...,...,...,...,...,...
670,/m/0k60,/m/has_value,0.736635,0.914300,0.788354
671,/m/038nv6,/m/has_value,0.214328,0.254681,0.032972
672,/m/09_2gj,/m/has_value,0.358785,0.626478,0.210967
673,/m/011k4g,/m/has_value,0.285680,0.350715,0.668902


In [245]:
acc = (len(merged[merged['literal'] > 0.5][merged[f'score {category_a}'] > merged[f'score {category_b}']]) + len(merged[merged['literal'] <= 0.5][merged[f'score {category_a}'] < merged[f'score {category_b}']])) / len(merged)

/var/folders/vc/8d10xrmj0xj3v8g5_6zpmmkm0000gn/T/ipykernel_3184/647973516.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  acc = (len(merged[merged['literal'] > 0.5][merged[f'score {category_a}'] > merged[f'score {category_b}']]) + len(merged[merged['literal'] <= 0.5][merged[f'score {category_a}'] < merged[f'score {category_b}']])) / len(merged)


todo: update formular

$ acc = | \lbrace   (s, \text{has value}, \text{high}) |  f((s,\text{has value}, \text{high})) > f((\text{has value}, \text{low}))  \land l > 0.5        \rbrace     |     $

In [246]:
acc

0.522962962962963

In [276]:
# filter dataframes to only contain the relevant entities
no_human_results = df_results[~df_results['head'].isin(human_uris.to_list())]
human_results = df_results[df_results['head'].isin(human_uris.to_list())]

In [277]:
# compute MR for human and non-human entities
mr_no_human = no_human_results[no_human_results['relation']==value_relation]['rank tail'].mean()
mr_human = human_results[human_results['relation']==value_relation]['rank tail'].mean()

mr_diff = mr_no_human - mr_human

todo: update formular
$MR_{diff} = \frac{\sum_{(s,p,o) \in G_{\neg human}}  r(s,p,o)    }{ |G_{\neg human}|}      -        \frac{\sum_{(s,p,o) \in G_{human}}  r(s,p,o)    }{ |G_{human}|}  $

In [279]:
mr_diff

5816.098246187365